In [ ]:
import requests
from pydriller import Repository
import pathlib
import textwrap
import datetime
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown

In [ ]:
import requests
import time

class GitHubHandler:
    def __init__(self, api_keys):
        self.api_keys = api_keys
        self.current_key_index = 0
        self.rate_limit_wait_time = 60
        self.max_retries = 3
        self.retryable_exceptions = (
            requests.exceptions.RequestException,
            requests.exceptions.ConnectionError,
            requests.exceptions.Timeout,
            requests.exceptions.HTTPError,
        )
        self.reset_times = {}

    def _key_handler(self):
        if len(self.api_keys) == 0:
            print("No API keys available")
            return None

        key = self.api_keys[self.current_key_index]
        self.current_key_index = (self.current_key_index + 1) % len(self.api_keys)
        return key

    def get_rate_limit_info(response):
        rate_limit_info = {
            "limit": int(response.headers["X-RateLimit-Limit"]),
            "remaining": int(response.headers["X-RateLimit-Remaining"]),
            "reset": int(response.headers["X-RateLimit-Reset"]),
        }
        return rate_limit_info

    def get(self, url, headers=None, params=None):
        if headers is None:
            headers = {}

        for attempt in range(1, self.max_retries + 1):
            try:
                headers['Authorization'] = f"token {self._key_handler()}"
                response = requests.get(url, headers=headers, params=params)
                response.raise_for_status()

                rate_limit_info = self.get_rate_limit_info(response)
                print(f"Rate limit info: {rate_limit_info}")
                if rate_limit_info['remaining'] == 0:
                    print(f"Rate limit reached. Waiting for {self.rate_limit_wait_time} seconds...")
                    time.sleep(self.rate_limit_wait_time)
                    continue

                if "GitHub-Authentication-Token-Expiration" in response.headers:
                    expiration_time = int(response.headers["GitHub-Authentication-Token-Expiration"])
                    self.reset_times[self.api_keys[self.current_key_index - 1]] = expiration_time
                    print(f"Key expires at {expiration_time}")

                current_key = self.api_keys[self.current_key_index - 1]
                if current_key in self.reset_times and time.time() > self.reset_times[current_key]:
                    del self.api_keys[self.current_key_index - 1]
                    del self.reset_times[current_key]
                    print(f"Key {current_key} expired. Removing.")
                    continue

                return response
            except self.retryable_exceptions as e:
                if attempt == self.max_retries:
                    print(f"Attempt {attempt}/{self.max_retries}: {e.__class__.__name__}")
                    if response:
                        print(f"Error details: {response.text}")
                    raise
                else:
                    print(f"Attempt {attempt}/{self.max_retries}: {e.__class__.__name__} occurred. Retrying...")

        return None

# Replace with your actual URL and API keys
url = "https://api.github.com/some/endpoint"
api_keys = []

data = GitHubHandler(api_keys).get(url)
print(data)


Attempt 1/3: HTTPError occurred. Retrying...
Attempt 2/3: HTTPError occurred. Retrying...
Attempt 3/3: HTTPError


HTTPError: 404 Client Error: Not Found for url: https://api.github.com/some/endpoint

In [ ]:
import requests
from pydriller import Repository

class GithubIssueFetcher:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "https://api.github.com"

    def fetch_issues(self, owner, repo):
        url = f"{self.base_url}/repos/{owner}/{repo}/issues"
        headers = {"Authorization": f"token {self.api_key}"}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch issues: {response.status_code}")
            return None

    def fetch_commits(self, owner, repo):
        commits = []
        for commit in Repository(f"https://github.com/{owner}/{repo}").traverse_commits():
            commits.append(commit)
        return commits

# Example usage:
if __name__ == "__main__":
    api_key = ""
    owner = "lcompilers"
    repo = "lpython"

    fetcher = GithubIssueFetcher(api_key)
    issues = fetcher.fetch_issues(owner, repo)
    if issues:
        print("Issues:")
        for issue in issues:
            print(f"Issue #{issue['number']}: {issue['title']}")

    commits = fetcher.fetch_commits(owner, repo)
    print("\nCommits:")
    for commit in commits:
        print(f"Commit: {commit.hash} by {commit.author.name} - {commit.msg}")


Streaming output truncated to the last 5000 lines.
Commit: 2b3358c3c6814b5a51a69cf81a7cdb0599c43a81 by Thirumalai-Shaktivel - Recognize type_ignore within the dictionaries
Commit: 4526b5d76ca87b0c7a322daf0825851fdf2fbf94 by Thirumalai-Shaktivel - Add tests and update the refs.
Commit: 11531bfe91df4614d594d6cd440886f959d2f7a7 by Thirumalai Shaktivel - Merge pull request #915 from Thirumalai-Shaktivel/fix_dict
Commit: 46a4d007255f6f9824a1cfa05bce5f2a5ab4316c by Naman Gera - Merge pull request #912 from namannimmo10/ifexp

Implement `IfExp` visitor in the LLVM backend
Commit: f68c56c292f7b554712015edf7a1f58e63c37410 by Thirumalai Shaktivel - Merge pull request #913 from Thirumalai-Shaktivel/run_tests
Commit: 527ba7eb4799c706b4c5630075575fa579266d9e by Ondřej Čertík - Merge pull request #914 from Smit-create/i-883

Implement for loop with slice/subscript objects
Commit: c4b1224d150a910c3d7aa1f34556f220e7fef17a by Ondřej Čertík - Port ASR from LFortran
Commit: 7472e7c26741ad178722bfad7db620

In [ ]:
# utils funcs
def json_dump(data, filename):
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)